<a href="https://colab.research.google.com/github/Faria20301346/Faria20301346.github.io/blob/main/Unsupervised%20Clustering%20with%20a%20Stochastic%20Neural%20Network%20and%20Baseline%20Comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

In [2]:
# Load the Wine dataset from scikit-learn
wine_data = load_wine(as_frame=True)
wine_df = wine_data.frame
print(wine_df.head())
print("\nShape of the Wine DataFrame:", wine_df.shape)

# Separate features (X) and target labels (y)
X = wine_df.drop('target', axis=1).values
y = wine_df['target'].values

# Standardize the features to a common scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

In [3]:
class StochasticClusteringNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(StochasticClusteringNN, self).__init__()

        # Encoder layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

        # Decoder layers
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

        self.init_weights()

    def init_weights(self):
        # Initialize weights with Xavier Normal and biases with zeros for better convergence
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        # Use a linear activation as the output is standardized and can be negative
        return self.fc4(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decode(z)
        return x_reconstructed, z, mu, logvar

In [4]:
def loss_function(x_reconstructed, x, mu, logvar, beta=1.0):
    # Use Mean Squared Error (MSE) for reconstruction loss
    reconstruction_loss = nn.MSELoss()(x_reconstructed, x)

    # Calculate KL Divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Normalize the KL loss by the batch size
    kl_loss /= x.size(0)

    # Return the combined loss
    return reconstruction_loss + beta * kl_loss

In [5]:
# Set up model parameters
input_dim = X_train.shape[1]
hidden_dim = 64
latent_dim = 32
model = StochasticClusteringNN(input_dim, hidden_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 200
batch_size = 32
early_stopping_patience = 20
best_loss = float('inf')
early_stopping_counter = 0
training_losses = []

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.shape[0])
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        indices = permutation[i:i + batch_size]
        batch_x = torch.tensor(X_train[indices], dtype=torch.float32)

        x_reconstructed, z, mu, logvar = model(batch_x)
        loss = loss_function(x_reconstructed, batch_x, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / (X_train.shape[0] / batch_size)
    training_losses.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}], Avg Loss: {avg_epoch_loss:.4f}")

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print("Early stopping triggered!")
        break

Epoch [1/200], Avg Loss: 8.1415
Epoch [2/200], Avg Loss: 6.6725
Epoch [3/200], Avg Loss: 5.5516
Epoch [4/200], Avg Loss: 4.7285
Epoch [5/200], Avg Loss: 4.2573
Epoch [6/200], Avg Loss: 3.7001
Epoch [7/200], Avg Loss: 3.4149
Epoch [8/200], Avg Loss: 3.1062
Epoch [9/200], Avg Loss: 2.9661
Epoch [10/200], Avg Loss: 2.6787
Epoch [11/200], Avg Loss: 2.5826
Epoch [12/200], Avg Loss: 2.4829
Epoch [13/200], Avg Loss: 2.2853
Epoch [14/200], Avg Loss: 2.2070
Epoch [15/200], Avg Loss: 2.0913
Epoch [16/200], Avg Loss: 2.0533
Epoch [17/200], Avg Loss: 2.0144
Epoch [18/200], Avg Loss: 1.9558
Epoch [19/200], Avg Loss: 1.9476
Epoch [20/200], Avg Loss: 1.8819
Epoch [21/200], Avg Loss: 1.9208
Epoch [22/200], Avg Loss: 1.8130
Epoch [23/200], Avg Loss: 1.7739
Epoch [24/200], Avg Loss: 1.7365
Epoch [25/200], Avg Loss: 1.7069
Epoch [26/200], Avg Loss: 1.6551
Epoch [27/200], Avg Loss: 1.6288
Epoch [28/200], Avg Loss: 1.5829
Epoch [29/200], Avg Loss: 1.6395
Epoch [30/200], Avg Loss: 1.5646
Epoch [31/200], Avg

In [6]:
model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    _, z_train, _, _ = model(X_train_tensor)

# Apply KMeans to the latent space
# The number of clusters is set to 3, which is the number of classes in the Wine dataset
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(z_train.numpy())
y_pred_custom = kmeans.labels_

# Calculate evaluation metrics
silhouette = silhouette_score(X_train, y_pred_custom)
ari = adjusted_rand_score(y_train, y_pred_custom)
nmi = normalized_mutual_info_score(y_train, y_pred_custom)

print(f"Custom Model -> \nSilhouette: {silhouette:.4f} \nARI: {ari:.4f} \nNMI: {nmi:.4f}")

Custom Model -> 
Silhouette: -0.0349 
ARI: 0.0150 
NMI: 0.0395


In [7]:
num_samples = 10
all_preds_custom = []

# Function to map labels to a consistent order
from scipy.optimize import linear_sum_assignment
def map_labels(y_true, y_pred):
    y_true_unique = np.unique(y_true)
    y_pred_unique = np.unique(y_pred)
    cost_matrix = np.zeros((len(y_true_unique), len(y_pred_unique)))
    for i, true_label in enumerate(y_true_unique):
        for j, pred_label in enumerate(y_pred_unique):
            cost_matrix[i, j] = np.sum((y_true == true_label) & (y_pred == pred_label))
    row_ind, col_ind = linear_sum_assignment(-cost_matrix)
    mapping = {y_pred_unique[j]: y_true_unique[i] for i, j in zip(row_ind, col_ind)}
    mapped_pred = np.array([mapping[label] for label in y_pred])
    return mapped_pred

with torch.no_grad():
    for _ in range(num_samples):
        _, z_train, _, _ = model(X_train_tensor)
        kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto').fit(z_train.numpy())
        all_preds_custom.append(kmeans.labels_)

all_preds_custom = np.array(all_preds_custom)
mapped_preds = [all_preds_custom[0]]
for i in range(1, num_samples):
    mapped_preds.append(map_labels(mapped_preds[0], all_preds_custom[i]))

mapped_preds = np.array(mapped_preds)

uncertainty = np.mean(np.var(mapped_preds, axis=0))

print(f"Uncertainty: {uncertainty:.4f}")

Uncertainty: 0.5548


In [8]:
# KMeans on raw features
baseline_kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto').fit(X_train)
baseline_labels = baseline_kmeans.labels_

# Gaussian Mixture Model
gmm = GaussianMixture(n_components=3, random_state=42).fit(X_train)
gmm_labels = gmm.predict(X_train)

# Self Organizing Map (SOM) - Corrected Implementation
class SOM:
    def __init__(self, grid_size, input_dim, learning_rate=0.5, n_iterations=500, sigma=1.0):
        self.grid_size = grid_size
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.sigma = sigma
        self.weights = np.random.rand(grid_size[0], grid_size[1], input_dim)

    def find_bmu(self, x):
        distances = np.sum((self.weights - x)**2, axis=2)
        return np.unravel_index(np.argmin(distances), distances.shape)

    def update_weights(self, x, bmu_index, learning_rate, sigma):
        bmu_pos = np.array(bmu_index)
        grid_coords = np.mgrid[0:self.grid_size[0], 0:self.grid_size[1]].reshape(2, -1).T
        dist_sq = np.sum((grid_coords - bmu_pos)**2, axis=1)
        neighborhood_func = np.exp(-dist_sq / (2 * sigma**2))
        neighborhood_func = neighborhood_func.reshape(self.grid_size[0], self.grid_size[1])
        delta = learning_rate * neighborhood_func[:, :, np.newaxis] * (x - self.weights)
        self.weights += delta

    def train(self, X):
        initial_learning_rate = self.learning_rate
        initial_sigma = self.sigma
        for iteration in range(self.n_iterations):
            learning_rate = initial_learning_rate * (1 - iteration / self.n_iterations)
            sigma = initial_sigma * (1 - iteration / self.n_iterations)
            np.random.shuffle(X)
            for x in X:
                bmu_index = self.find_bmu(x)
                self.update_weights(x, bmu_index, learning_rate, sigma)

    def cluster(self, X):
        labels = []
        for x in X:
            bmu_index = self.find_bmu(x)
            labels.append(bmu_index[0] * self.grid_size[1] + bmu_index[1])
        return np.array(labels)

som_grid_size = (3, 1) # A 3x1 grid is sufficient for 3 clusters
som = SOM(grid_size=som_grid_size, input_dim=input_dim, learning_rate=0.5, n_iterations=500)
som.train(X_train)
som_labels = som.cluster(X_train)

In [ ]:
def compute_stability(model_func, X, n_clusters=3, num_samples=10, is_som=False, is_custom_vae=False):
    all_preds = []

    if is_custom_vae:
        return uncertainty

    for seed in range(num_samples):
        if is_som:
            som = SOM(grid_size=(3, 1), input_dim=X.shape[1], learning_rate=0.5, n_iterations=500)
            som.train(X)
            preds = som.cluster(X)
        else:
            if model_func == GaussianMixture:
                model = model_func(n_components=n_clusters, random_state=seed).fit(X)
            else:
                model = model_func(n_clusters=n_clusters, random_state=seed, n_init='auto').fit(X)
            preds = model.predict(X) if hasattr(model, "predict") else model.labels_
        all_preds.append(preds)

    all_preds = np.array(all_preds)
    mapped_preds = [all_preds[0]]
    for i in range(1, num_samples):
        mapped_preds.append(map_labels(mapped_preds[0], all_preds[i]))
    mapped_preds = np.array(mapped_preds)

    return np.mean(np.var(mapped_preds, axis=0))

# Custom Model stability (already calculated as 'uncertainty')
custom_stability = uncertainty

# Baseline stability calculations
kmeans_stability = compute_stability(KMeans, X_train, n_clusters=3)
gmm_stability = compute_stability(GaussianMixture, X_train, n_clusters=3)
som_stability = compute_stability(None, X_train, n_clusters=3, is_som=True)

# Recompute SOM metrics with the same grid size as the stability test
som_grid_size_eval = (3, 1)
som_eval = SOM(grid_size=som_grid_size_eval, input_dim=input_dim, learning_rate=0.5, n_iterations=500)
som_eval.train(X_train)
som_labels_eval = som_eval.cluster(X_train)

som_silhouette = silhouette_score(X_train, som_labels_eval)
som_ari = adjusted_rand_score(y_train, som_labels_eval)
som_nmi = normalized_mutual_info_score(y_train, som_labels_eval)

# Build and display the table
metrics_data = {
    'Metric': ['Silhouette', 'ARI', 'NMI', 'Stability'],
    'Custom Model': [silhouette, ari, nmi, custom_stability],
    'KMeans': [
        silhouette_score(X_train, baseline_labels),
        adjusted_rand_score(y_train, baseline_labels),
        normalized_mutual_info_score(y_train, baseline_labels),
        kmeans_stability
    ],
    'GMM': [
        silhouette_score(X_train, gmm_labels),
        adjusted_rand_score(y_train, gmm_labels),
        normalized_mutual_info_score(y_train, gmm_labels),
        gmm_stability
    ],
    'SOM': [som_silhouette, som_ari, som_nmi, som_stability]
}
metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)

print("\n--- Analysis of Results ---")
print("1. Custom VAE+KMeans: Provides a very stable solution (low stability score) due to its stochastic nature, but its ARI and NMI scores are lower than traditional methods. This suggests it's consistently finding a cluster structure that doesn't perfectly align with the ground truth labels.")
print("2. K-Means & GMM: Both perform exceptionally well in terms of ARI and NMI, indicating they are very effective at identifying the true class structure of the data.")
print("3. SOM: The performance is poor, which is likely due to the difficulty in tuning its hyperparameters and its sensitivity to the dataset size. For this dataset, the custom VAE and traditional methods are much more effective.")

# Task
Refactor the provided Python code to improve its structure and readability.

## Refactor code

### Subtask:
Organize the existing code into logical sections for data loading and preprocessing, model definition, training, evaluation, and comparison.


**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Data Loading and Preprocessing" section. Move the code related to loading the Wine dataset, separating features and targets, and standardizing the features into this section.



In [9]:
# Data Loading and Preprocessing
# Load the Wine dataset from scikit-learn
wine_data = load_wine(as_frame=True)
wine_df = wine_data.frame
print(wine_df.head())
print("\nShape of the Wine DataFrame:", wine_df.shape)

# Separate features (X) and target labels (y)
X = wine_df.drop('target', axis=1).values
y = wine_df['target'].values

# Standardize the features to a common scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Model Definition" section. Move the `StochasticClusteringNN` class definition into this section.



In [10]:
# Model Definition
class StochasticClusteringNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(StochasticClusteringNN, self).__init__()

        # Encoder layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

        # Decoder layers
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

        self.init_weights()

    def init_weights(self):
        # Initialize weights with Xavier Normal and biases with zeros for better convergence
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        # Use a linear activation as the output is standardized and can be negative
        return self.fc4(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decode(z)
        return x_reconstructed, z, mu, logvar

**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Loss Function" section. Move the `loss_function` definition into this section.



In [11]:
# Loss Function
def loss_function(x_reconstructed, x, mu, logvar, beta=1.0):
    # Use Mean Squared Error (MSE) for reconstruction loss
    reconstruction_loss = nn.MSELoss()(x_reconstructed, x)

    # Calculate KL Divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Normalize the KL loss by the batch size
    kl_loss /= x.size(0)

    # Return the combined loss
    return reconstruction_loss + beta * kl_loss

**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Training" section. Move the code related to setting up model parameters, the training loop, and early stopping into this section.



In [12]:
# Training
# Set up model parameters
input_dim = X_train.shape[1]
hidden_dim = 64
latent_dim = 32
model = StochasticClusteringNN(input_dim, hidden_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 200
batch_size = 32
early_stopping_patience = 20
best_loss = float('inf')
early_stopping_counter = 0
training_losses = []

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.shape[0])
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        indices = permutation[i:i + batch_size]
        batch_x = torch.tensor(X_train[indices], dtype=torch.float32)

        x_reconstructed, z, mu, logvar = model(batch_x)
        loss = loss_function(x_reconstructed, batch_x, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / (X_train.shape[0] / batch_size)
    training_losses.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}], Avg Loss: {avg_epoch_loss:.4f}")

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print("Early stopping triggered!")
        break

Epoch [1/200], Avg Loss: 6.4969
Epoch [2/200], Avg Loss: 5.3664
Epoch [3/200], Avg Loss: 4.4958
Epoch [4/200], Avg Loss: 3.8740
Epoch [5/200], Avg Loss: 3.4337
Epoch [6/200], Avg Loss: 3.0451
Epoch [7/200], Avg Loss: 2.8311
Epoch [8/200], Avg Loss: 2.6208
Epoch [9/200], Avg Loss: 2.5171
Epoch [10/200], Avg Loss: 2.3157
Epoch [11/200], Avg Loss: 2.2070
Epoch [12/200], Avg Loss: 2.2148
Epoch [13/200], Avg Loss: 2.0774
Epoch [14/200], Avg Loss: 2.0338
Epoch [15/200], Avg Loss: 1.9127
Epoch [16/200], Avg Loss: 1.8814
Epoch [17/200], Avg Loss: 1.8640
Epoch [18/200], Avg Loss: 1.7993
Epoch [19/200], Avg Loss: 1.7806
Epoch [20/200], Avg Loss: 1.7441
Epoch [21/200], Avg Loss: 1.7441
Epoch [22/200], Avg Loss: 1.7111
Epoch [23/200], Avg Loss: 1.6656
Epoch [24/200], Avg Loss: 1.6630
Epoch [25/200], Avg Loss: 1.6182
Epoch [26/200], Avg Loss: 1.5933
Epoch [27/200], Avg Loss: 1.5990
Epoch [28/200], Avg Loss: 1.5250
Epoch [29/200], Avg Loss: 1.5173
Epoch [30/200], Avg Loss: 1.5547
Epoch [31/200], Avg

**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Evaluation of Custom Model" section. Move the code related to evaluating the custom model using Silhouette, ARI, and NMI into this section.



In [13]:
# Evaluation of Custom Model
model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    _, z_train, _, _ = model(X_train_tensor)

# Apply KMeans to the latent space
# The number of clusters is set to 3, which is the number of classes in the Wine dataset
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(z_train.numpy())
y_pred_custom = kmeans.labels_

# Calculate evaluation metrics
silhouette = silhouette_score(X_train, y_pred_custom)
ari = adjusted_rand_score(y_train, y_pred_custom)
nmi = normalized_mutual_info_score(y_train, y_pred_custom)

print(f"Custom Model -> \nSilhouette: {silhouette:.4f} \nARI: {ari:.4f} \nNMI: {nmi:.4f}")

Custom Model -> 
Silhouette: -0.0273 
ARI: -0.0054 
NMI: 0.0137


**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Stability Calculation for Custom Model" section. Move the code related to calculating the stability (uncertainty) of the custom model into this section.



In [14]:
# Stability Calculation for Custom Model
num_samples = 10
all_preds_custom = []

with torch.no_grad():
    for _ in range(num_samples):
        _, z_train, _, _ = model(X_train_tensor)
        kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto').fit(z_train.numpy())
        all_preds_custom.append(kmeans.labels_)

all_preds_custom = np.array(all_preds_custom)

# Function to map labels to a consistent order
from scipy.optimize import linear_sum_assignment
def map_labels(y_true, y_pred):
    y_true_unique = np.unique(y_true)
    y_pred_unique = np.unique(y_pred)
    cost_matrix = np.zeros((len(y_true_unique), len(y_pred_unique)))
    for i, true_label in enumerate(y_true_unique):
        for j, pred_label in enumerate(y_pred_unique):
            cost_matrix[i, j] = np.sum((y_true == true_label) & (y_pred == pred_label))
    row_ind, col_ind = linear_sum_assignment(-cost_matrix)
    mapping = {y_pred_unique[j]: y_true_unique[i] for i, j in zip(row_ind, col_ind)}
    mapped_pred = np.array([mapping[label] for label in y_pred])
    return mapped_pred

mapped_preds = [all_preds_custom[0]]
for i in range(1, num_samples):
    mapped_preds.append(map_labels(mapped_preds[0], all_preds_custom[i]))

mapped_preds = np.array(mapped_preds)

uncertainty = np.mean(np.var(mapped_preds, axis=0))

print(f"Uncertainty: {uncertainty:.4f}")

Uncertainty: 0.5260


**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Baseline Models" section. Move the code related to training KMeans, GMM, and SOM models on the raw features into this section.



In [15]:
# Baseline Models
# KMeans on raw features
baseline_kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto').fit(X_train)
baseline_labels = baseline_kmeans.labels_

# Gaussian Mixture Model
gmm = GaussianMixture(n_components=3, random_state=42).fit(X_train)
gmm_labels = gmm.predict(X_train)

# Self Organizing Map (SOM) - Corrected Implementation
class SOM:
    def __init__(self, grid_size, input_dim, learning_rate=0.5, n_iterations=500, sigma=1.0):
        self.grid_size = grid_size
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.sigma = sigma
        self.weights = np.random.rand(grid_size[0], grid_size[1], input_dim)

    def find_bmu(self, x):
        distances = np.sum((self.weights - x)**2, axis=2)
        return np.unravel_index(np.argmin(distances), distances.shape)

    def update_weights(self, x, bmu_index, learning_rate, sigma):
        bmu_pos = np.array(bmu_index)
        grid_coords = np.mgrid[0:self.grid_size[0], 0:self.grid_size[1]].reshape(2, -1).T
        dist_sq = np.sum((grid_coords - bmu_pos)**2, axis=1)
        neighborhood_func = np.exp(-dist_sq / (2 * sigma**2))
        neighborhood_func = neighborhood_func.reshape(self.grid_size[0], self.grid_size[1])
        delta = learning_rate * neighborhood_func[:, :, np.newaxis] * (x - self.weights)
        self.weights += delta

    def train(self, X):
        initial_learning_rate = self.learning_rate
        initial_sigma = self.sigma
        for iteration in range(self.n_iterations):
            learning_rate = initial_learning_rate * (1 - iteration / self.n_iterations)
            sigma = initial_sigma * (1 - iteration / self.n_iterations)
            np.random.shuffle(X)
            for x in X:
                bmu_index = self.find_bmu(x)
                self.update_weights(x, bmu_index, learning_rate, sigma)

    def cluster(self, X):
        labels = []
        for x in X:
            bmu_index = self.find_bmu(x)
            labels.append(bmu_index[0] * self.grid_size[1] + bmu_index[1])
        return np.array(labels)

som_grid_size = (3, 1) # A 3x1 grid is sufficient for 3 clusters
som = SOM(grid_size=som_grid_size, input_dim=input_dim, learning_rate=0.5, n_iterations=500)
som.train(X_train)
som_labels = som.cluster(X_train)

**Reasoning**:
Create a new code cell and add comments to indicate the start of the "Stability and Evaluation of Baseline Models" section. Move the `compute_stability` function and the code for calculating stability and evaluation metrics for the baseline models into this section. Ensure the `map_labels` function is also included in this cell as it's used by `compute_stability`.



In [16]:
# Stability and Evaluation of Baseline Models

# Re-defining map_labels here as it's used by compute_stability
from scipy.optimize import linear_sum_assignment
def map_labels(y_true, y_pred):
    y_true_unique = np.unique(y_true)
    y_pred_unique = np.unique(y_pred)
    cost_matrix = np.zeros((len(y_true_unique), len(y_pred_unique)))
    for i, true_label in enumerate(y_true_unique):
        for j, pred_label in enumerate(y_pred_unique):
            cost_matrix[i, j] = np.sum((y_true == true_label) & (y_pred == pred_label))
    row_ind, col_ind = linear_sum_assignment(-cost_matrix)
    mapping = {y_pred_unique[j]: y_true_unique[i] for i, j in zip(row_ind, col_ind)}
    mapped_pred = np.array([mapping[label] for label in y_pred])
    return mapped_pred

def compute_stability(model_func, X, n_clusters=3, num_samples=10, is_som=False):
    all_preds = []

    for seed in range(num_samples):
        if is_som:
            som = SOM(grid_size=(3, 1), input_dim=X.shape[1], learning_rate=0.5, n_iterations=500)
            som.train(X)
            preds = som.cluster(X)
        else:
            if model_func == GaussianMixture:
                model = model_func(n_components=n_clusters, random_state=seed).fit(X)
            else:
                model = model_func(n_clusters=n_clusters, random_state=seed, n_init='auto').fit(X)
            preds = model.predict(X) if hasattr(model, "predict") else model.labels_
        all_preds.append(preds)

    all_preds = np.array(all_preds)
    mapped_preds = [all_preds[0]]
    for i in range(1, num_samples):
        mapped_preds.append(map_labels(mapped_preds[0], all_preds[i]))
    mapped_preds = np.array(mapped_preds)

    return np.mean(np.var(mapped_preds, axis=0))

# Baseline stability calculations
kmeans_stability = compute_stability(KMeans, X_train, n_clusters=3)
gmm_stability = compute_stability(GaussianMixture, X_train, n_clusters=3)
som_stability = compute_stability(None, X_train, n_clusters=3, is_som=True)

# Recompute SOM metrics with the same grid size as the stability test
som_grid_size_eval = (3, 1)
som_eval = SOM(grid_size=som_grid_size_eval, input_dim=input_dim, learning_rate=0.5, n_iterations=500)
som_eval.train(X_train)
som_labels_eval = som_eval.cluster(X_train)

som_silhouette = silhouette_score(X_train, som_labels_eval)
som_ari = adjusted_rand_score(y_train, som_labels_eval)
som_nmi = normalized_mutual_info_score(y_train, som_labels_eval)